# Percent Vegetation

This notebook uses ExGR-Zero-Threshold created in "2.2_Otsu_Thresh_Zero.ipynb", the prepared images ceated in "0.2_Image_Crop.ipynb", and the vegetation indicies created in "1.0_Vegetation_Index.ipynb" to summarize vegetation cover. The reason why ExGR-Zero-Threshold is used is because it had the highest overall accuracy. First, a dataframe is created using "Oat_Data.csv". Then, percent vegetation is found  by taking the average of each classified image (total vegetation/ area). The normalized green is found from the prepared images and clipped to the classified vegetation. EXG and EXGR indicies are also clipped to the classified vegetation. Finally, the average is found by taking the sum of the value and dividing it by the sum of classified vegetation.

Luke Rosen 5/19/2022

In [1]:
import os
import numpy as np
import glob
import skimage
from skimage.io import imread, imsave, imshow
import pandas as pd
from datetime import datetime

In [2]:
#Opening and moving old dataframe.
Oat_Data = pd.read_csv("./Oat_Data.csv")
if not os.path.exists("./Archive"):
    os.mkdir("./Archive")
Oat_Data.to_csv("./Archive/Oat_Data_" + str(datetime.today().month) + "_" + str(datetime.today().day) + ".csv", index = False)


In [3]:
Oat_Data

,Image,Plot,Date_Index,Date,Variety,Cover,Green_Avg,EXG_Avg_Image,EXGR_Avg_Image,GLI_Avg_Image,...,GLI_Local_Thresh,VARI_Local_Thresh,EXG_Global_Thresh,EXGR_Global_Thresh,GLI_Global_Thresh,VARI_Global_Thresh,Vegetation_Fraction,Green_Avg_Vegetation,EXG_Avg_Vegetation,EXGR_Avg_Vegetation
0,101A_20210708,101A,20210708,7/8/2021,Oat,Fallow,0.345924,0.037808,-0.154819,0.024490,...,-0.324219,-0.164062,0.306641,0.205078,0.160156,-0.089844,0.103864,0.414502,0.243505,0.161543
1,101B_20210708,101B,20210708,7/8/2021,Oat,Post,0.343246,0.029749,-0.198514,0.017911,...,-0.300781,-0.214844,0.306641,0.205078,0.160156,-0.089844,0.090492,0.417833,0.253498,0.147854
2,101C_20210708,101C,20210708,7/8/2021,Oat,Under,0.345618,0.036861,-0.183141,0.023596,...,-0.300781,-0.185547,0.306641,0.205078,0.160156,-0.089844,0.099949,0.417461,0.252382,0.150127
3,102A_20210708,102A,20210708,7/8/2021,Oat,Fallow,0.341648,0.024950,-0.217639,0.014854,...,-0.269531,-0.248047,0.306641,0.205078,0.160156,-0.089844,0.064850,0.414505,0.243515,0.127273
4,102B_20210708,102B,20210708,7/8/2021,Oat,Post,0.343716,0.031162,-0.207396,0.017689,...,-0.316406,-0.294922,0.306641,0.205078,0.160156,-0.089844,0.102550,0.414900,0.244701,0.140041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,409B_20211025,409B,20211025,10/25/2021,Oat,Fallow,0.324957,-0.025017,-0.260596,-0.029944,...,-0.347656,-0.462891,0.306641,0.205078,0.160156,-0.089844,0.101028,0.427930,0.283790,0.266255
836,409C_20211025,409C,20211025,10/25/2021,Oat,Post,0.403345,0.210126,0.108752,0.131284,...,0.191406,-0.117188,0.306641,0.205078,0.160156,-0.089844,0.542867,0.468458,0.405373,0.420653
837,410A_20211025,410A,20211025,10/25/2021,Oat,Under,0.486970,0.460969,0.502542,0.290155,...,0.371094,0.191406,0.306641,0.205078,0.160156,-0.089844,0.875943,0.508097,0.524291,0.604697
838,410B_20211025,410B,20211025,10/25/2021,Oat,Fallow,0.336206,0.008785,-0.204253,-0.007075,...,-0.339844,-0.480469,0.306641,0.205078,0.160156,-0.089844,0.165693,0.444000,0.332001,0.325336


In [4]:
count = 0
Image = []

Plot = []

Date = []

Variety = [] 

Cover = [] 

Frac_Veg = []

Veg_Mean_Green =[]

Veg_Mean_EXGR = []

Veg_Mean_EXG = []

Veg_Mean_GLI = []

Veg_Mean_VARI = []
print("Starting Vegetation Fraction...\n")
for index,  row in Oat_Data.iterrows():
    if row.Plot.startswith("106") or row.Plot.startswith("203") or row.Plot.startswith("303") or row.Plot.startswith("404"): ##Conditions of variety
        crop = "Soy"
        Variety.append(crop)
    else:
        crop = "Oat"
        Variety.append(crop)
    if row.Plot.startswith("1") and row.Plot.endswith("A") or not row.Plot.startswith("1") and row.Plot.endswith("B"): ##Conditions of cover
        cover = "Fallow"
        Cover.append(cover)
    elif row.Plot.startswith("1") and row.Plot.endswith("C") or row.Plot.startswith("3") and row.Plot.endswith("C") or row.Plot.startswith("2") and row.Plot.endswith("A") or row.Plot.startswith("4") and row.Plot.endswith("A"):
        cover = "Under"
        Cover.append(cover)
    else:
        cover = "Post"
        Cover.append(cover)
    classified = imread("./EXGR/Thresh/" + row.Plot + "_" + str(row.Date_Index) + "_EXGR_Thresh_Zero.tif")
    Image.append(row.Image)
    Plot.append(row.Plot)
    Date.append(str(row.Date_Index))
    percent = classified.mean()
    Frac_Veg.append(percent)
    
    with np.errstate(divide = "ignore", invalid = "ignore"):
        ref = imread("./images_prepared/" + row.Plot + "_" + str(row.Date_Index) + ".tif") #plot_date.tif

        Red_c = ref[:,:,0] * 1.0     ## Red color band as float

        Green_c = ref[:,:,1] * 1.0   ## Green color band as float

        Blue_c = ref[:,:,2] * 1.0    ## Blue color band as fload 

        Green_n = Green_c/(Red_c + Green_c + Blue_c) ## Normalize Green
        Green_n[np.isnan(Green_n)] = 0
        Green_n[np.isinf(Green_n)] = 0

        Green_veg = Green_n * classified                  ##Turns nonvegetation to zero
        Green_ave_Veg =  Green_veg.sum()/classified.sum() ##Average green 
        Veg_Mean_Green.append(Green_ave_Veg)              ##Appending verage green
    ind_EXGR = imread("./EXGR/" + row.Plot + "_" + str(row.Date_Index) + "_EXGR.tif") ##plot_date_EXGR.tif
    ind_EXGR_veg = ind_EXGR * classified                                              #Turns nonvegetation to zero
    
    ind_EXG = imread("./EXG/" + row.Plot + "_" + str(row.Date_Index) + "_EXG.tif")    ##plot_date_EXG.tif
    ind_EXG_veg = ind_EXG * classified                                                ##Turns nonvegetation to zero
    
    ind_GLI = imread("./GLI/" + row.Plot + "_" + str(row.Date_Index) + "_GLI.tif")    ##plot_date_GLI.tif
    ind_GLI_veg = ind_GLI * classified                                                ##Turns nonvegetation to zero
    
    ind_VARI = imread("./VARI/" + row.Plot + "_" + str(row.Date_Index) + "_VARI.tif")    ##plot_date_VARI.tif
    ind_VARI_veg = ind_VARI * classified                                                ##Turns nonvegetation to zero
    
    EXGR_mean_veg = ind_EXGR_veg.sum()/classified.sum()                  ##sum of index clipped/area of 
    Veg_Mean_EXGR.append(EXGR_mean_veg)                                  ##appending average
    
    EXG_mean_veg = ind_EXG_veg.sum()/classified.sum()
    Veg_Mean_EXG.append(EXG_mean_veg)
    
    GLI_mean_veg = ind_GLI_veg.sum()/classified.sum()
    Veg_Mean_GLI.append(GLI_mean_veg)
    
    VARI_mean_veg = ind_VARI_veg.sum()/classified.sum()
    Veg_Mean_VARI.append(VARI_mean_veg)
    if count % 42 == 0:
        print("Done with " + row.Plot + "_" + str(row.Date_Index) + "...\n")
    count = count +1
print("Creating Dataframe...\n")
Percent_Veg = pd.DataFrame({"Image":Image,
                           "Plot":Plot,
                           "Date":Date,
                            "Variety":Variety,
                            "Cover":Cover,
                           "Vegetation_Fraction":Frac_Veg,
                           "Green_Avg_Vegetation": Veg_Mean_Green,
                           "EXG_Avg_Vegetation": Veg_Mean_EXG,
                           "EXGR_Avg_Vegetation": Veg_Mean_EXGR,
                           "GLI_Avg_Vegetation": Veg_Mean_GLI,
                           "VARI_Avg_Vegetation":Veg_Mean_VARI})
print("Vegetation Fraction Complete!!!")


Starting Vegetation Fraction...

Done with 101A_20210708...

Done with 205A_20210708...

Done with 309A_20210708...

Done with 103A_20210821...

Done with 207A_20210821...

Done with 401A_20210821...

Done with 105A_20210909...

Done with 209A_20210909...

Done with 403A_20210909...

Done with 107A_20210929...

Done with 301A_20210929...

Done with 405A_20210929...

Done with 109A_20211005...

Done with 303A_20211005...

Done with 407A_20211005...

Done with 201A_20211015...

Done with 305A_20211015...

Done with 409A_20211015...

Done with 203A_20211025...

Done with 307A_20211025...

Creating Dataframe...

Vegetation Fraction Complete!!!


In [5]:
Percent_Veg

,Image,Plot,Date,Variety,Cover,Vegetation_Fraction,Green_Avg_Vegetation,EXG_Avg_Vegetation,EXGR_Avg_Vegetation,GLI_Avg_Vegetation,VARI_Avg_Vegetation
0,101A_20210708,101A,20210708,Oat,Fallow,0.103864,0.414502,0.243505,0.161543,0.165699,0.110431
1,101B_20210708,101B,20210708,Oat,Post,0.090492,0.417833,0.253498,0.147854,0.173700,0.073228
2,101C_20210708,101C,20210708,Oat,Under,0.099949,0.417461,0.252382,0.150127,0.173104,0.077935
3,102A_20210708,102A,20210708,Oat,Fallow,0.064850,0.414505,0.243515,0.127273,0.167610,0.060665
4,102B_20210708,102B,20210708,Oat,Post,0.102550,0.414900,0.244701,0.140041,0.167746,0.074355
...,...,...,...,...,...,...,...,...,...,...,...
835,409B_20211025,409B,20211025,Oat,Fallow,0.101028,0.427930,0.283790,0.266255,0.186371,0.212816
836,409C_20211025,409C,20211025,Oat,Post,0.542867,0.468458,0.405373,0.420653,0.262521,0.229079
837,410A_20211025,410A,20211025,Oat,Under,0.875943,0.508097,0.524291,0.604697,0.330985,0.298510
838,410B_20211025,410B,20211025,Oat,Fallow,0.165693,0.444000,0.332001,0.325336,0.215333,0.212896


In [6]:
Percent_Veg.to_csv("./Percent_Vegetation.csv", index = False)

In [7]:
Oat_Data["Vegetation_Fraction"] = Percent_Veg["Vegetation_Fraction"]
Oat_Data["Green_Avg_Vegetation"] = Percent_Veg["Green_Avg_Vegetation"]
Oat_Data["EXG_Avg_Vegetation"] = Percent_Veg["EXG_Avg_Vegetation"]
Oat_Data["EXGR_Avg_Vegetation"]= Percent_Veg["EXGR_Avg_Vegetation"]
Oat_Data["GLI_Avg_Vegetation"]= Percent_Veg["GLI_Avg_Vegetation"]
Oat_Data["VARI_Avg_Vegetation"]= Percent_Veg["VARI_Avg_Vegetation"]

In [8]:
Oat_Data.to_csv("Oat_Data.csv", index = False)

This notebook is complete. The final notebook will summarize the data into graphs. 